# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846885379836                   -0.70    4.8
  2   -7.852324561729       -2.26       -1.53    1.0
  3   -7.852615305459       -3.54       -2.56    1.2
  4   -7.852645775759       -4.52       -2.88    2.5
  5   -7.852646476506       -6.15       -3.19    1.0
  6   -7.852646676680       -6.70       -4.05    1.2
  7   -7.852646686652       -8.00       -5.17    2.0
  8   -7.852646686725      -10.14       -5.43    2.0
  9   -7.852646686730      -11.30       -6.12    1.0
 10   -7.852646686730      -12.50       -6.61    1.5
 11   -7.852646686730      -13.51       -7.19    1.2
 12   -7.852646686730      -14.45       -7.89    1.8
 13   -7.852646686730   +  -14.75       -8.30    2.0
 14   -7.852646686730      -15.05       -9.62    1.2
 15   -7.852646686730   +  -15.05      -10.23    3.2
 16   -7.852646686730      -14.75      -10.21    1.2
 17   -7.852646686730   +  -14.45      -10.93 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846906697141                   -0.70           4.8
  2   -7.852555546977       -2.25       -1.64   0.80    2.0
  3   -7.852639440317       -4.08       -2.74   0.80    1.0
  4   -7.852646506362       -5.15       -3.36   0.80    2.0
  5   -7.852646679478       -6.76       -4.22   0.80    1.5
  6   -7.852646686584       -8.15       -4.76   0.80    1.8
  7   -7.852646686721       -9.86       -5.65   0.80    1.5
  8   -7.852646686730      -11.06       -6.35   0.80    2.2
  9   -7.852646686730      -12.75       -7.54   0.80    1.5
 10   -7.852646686730      -14.01       -8.27   0.80    2.2
 11   -7.852646686730   +  -14.75       -9.59   0.80    1.5
 12   -7.852646686730   +  -15.05       -9.66   0.80    3.0
 13   -7.852646686730      -14.75      -10.59   0.80    1.0
 14   -7.852646686730   +    -Inf      -11.57   0.80    1.2
 15   -7.852646686730      -14.75      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.402373e+01     3.558577e+00
 * time: 0.4378511905670166
     1     1.630955e+00     1.795456e+00
 * time: 0.6421639919281006
     2    -1.281718e+00     2.173860e+00
 * time: 0.6689810752868652
     3    -3.661604e+00     1.757000e+00
 * time: 0.7074990272521973
     4    -5.104503e+00     1.456765e+00
 * time: 0.7459001541137695
     5    -6.665368e+00     8.749559e-01
 * time: 0.7849199771881104
     6    -7.268133e+00     3.990410e-01
 * time: 0.8238141536712646
     7    -7.574643e+00     2.199881e-01
 * time: 0.8510210514068604
     8    -7.728537e+00     1.250909e-01
 * time: 0.8776991367340088
     9    -7.785921e+00     1.132267e-01
 * time: 0.9047801494598389
    10    -7.817256e+00     7.059717e-02
 * time: 0.9321310520172119
    11    -7.836621e+00     5.548029e-02
 * time: 0.9592440128326416
    12    -7.846166e+00     3.367398e-02
 * time: 0.9858860969543457
    13    -7.850584e+00     3.111255e-02
 * time: 1.0126090049

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846857262807                   -0.70    4.8
  2   -7.852322661705       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686713                   -2.55
  2   -7.852646686730      -10.78       -6.03
  3   -7.852646686730   +  -15.05      -12.71


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.3091371167458254e-13
|ρ_newton - ρ_scfv| = 9.599995169440409e-14
|ρ_newton - ρ_dm|   = 1.008511945871294e-9
